# Data ETL

## Imports and Setup

In [ ]:
import pandas as pd
import openpyxl
from datetime import datetime
import os

# Path to your Excel file
file_path = 'data/raw/Bloomberg_Data.xlsx'

# Define which sheets use column C instead of column B
use_column_c = [
    "US_Building_Permits",
    "US _BP_Single_Housing",
    "US_Housing_Start",
    "US_New_Home_Sales",
    "US_Existing_Home _Sales",
    "US Existing_Single_Home_Sales",
    "CAD_Housing_Start"
]

# Define sheets to ignore
sheets_to_ignore = [
    "US_Population_Growth_Rate_Bloom"
]

## Function Definitions

In [ ]:
def normalize_date(date):
    """Normalize date to end of month, with special handling for quarterly data"""
    if pd.isna(date):
        return None
    
    # Convert to datetime if not already
    if not isinstance(date, datetime):
        date = pd.to_datetime(date)
    
    # Get the last day of the month
    year = date.year
    month = date.month
    
    # Create end of month date
    if month == 12:
        end_of_month = datetime(year, 12, 31)
    else:
        end_of_month = datetime(year, month + 1, 1) - pd.Timedelta(days=1)
    
    return end_of_month.date()

def normalize_quarterly_date(date):
    """Normalize quarterly date to end of quarter"""
    if pd.isna(date):
        return None
    
    # Convert to datetime if not already
    if not isinstance(date, datetime):
        date = pd.to_datetime(date)
    
    year = date.year
    month = date.month
    
    # Map to end of quarter
    if month in [1, 2, 3]:  # Q1
        return datetime(year, 3, 31).date()
    elif month in [4, 5, 6]:  # Q2
        return datetime(year, 6, 30).date()
    elif month in [7, 8, 9]:  # Q3
        return datetime(year, 9, 30).date()
    else:  # Q4
        return datetime(year, 12, 31).date()

def extract_sheet_data(file_path, sheet_name, use_col_c):
    """Extract data from a specific sheet with improved data cleaning"""
    # Determine which column to use
    data_column = 'C' if sheet_name in use_col_c else 'B'
    
    # Read the sheet starting from row 7 (index 6 in pandas)
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Extract dates from column A and values from the appropriate column
    # Row 7 in Excel is index 6 in pandas (0-indexed)
    dates = df.iloc[6:, 0]  # Column A, starting from row 7
    
    if data_column == 'C':
        values = df.iloc[6:, 2]  # Column C
    else:
        values = df.iloc[6:, 1]  # Column B
    
    # Create a temporary dataframe
    temp_df = pd.DataFrame({
        'date': dates,
        'value': values
    })
    
    # Remove rows where value is NaN or empty
    temp_df = temp_df.dropna(subset=['value'])
    
    # Remove rows where date is NaN
    temp_df = temp_df.dropna(subset=['date'])
    
    # Determine the appropriate date normalization based on sheet name
    # Population growth data needs special quarterly normalization
    if sheet_name == 'US_Population_Growth_Rate_FRED':
        temp_df['date'] = temp_df['date'].apply(normalize_quarterly_date)
    else:
        temp_df['date'] = temp_df['date'].apply(normalize_date)
    
    # Remove any rows where date normalization failed
    temp_df = temp_df.dropna(subset=['date'])
    
    # Rename value column to sheet name
    temp_df = temp_df.rename(columns={'value': sheet_name})
    
    return temp_df

def is_row_worth_keeping(row, important_columns, min_important_values=5):
    """
    Determine if a row is worth keeping based on the number of important values.
    A row is worth keeping if it has at least min_important_values non-null values
    in important columns (excluding CPI-only rows).
    """
    # Count non-null values in important columns
    non_null_count = row[important_columns].notna().sum()
    
    # Special case: if the row only has CPI data, drop it
    if non_null_count == 0 and row.get('US_CPI') is not None:
        return False
    
    # Keep rows with sufficient important data
    return non_null_count >= min_important_values

## Data Extraction

In [ ]:
# Load the Excel file to get all sheet names
excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names

print(f"Found {len(sheet_names)} sheets in the Excel file\n")

# Extract data from all sheets
all_dataframes = []

for sheet_name in sheet_names:
    # Skip ignored sheets
    if sheet_name in sheets_to_ignore:
        print(f"Skipping: {sheet_name} (ignored)")
        continue
    
    print(f"Processing: {sheet_name}...", end=' ')
    try:
        df = extract_sheet_data(file_path, sheet_name, use_column_c)
        all_dataframes.append(df)
        print(f"✓ ({len(df)} data points)")
    except Exception as e:
        print(f"✗ Error: {e}")


## Data Merging

In [ ]:
# Merge all dataframes on the date column
print("\nMerging all data into master dataframe...")

master_df = all_dataframes[0]
for df in all_dataframes[1:]:
    master_df = master_df.merge(df, on='date', how='outer')

# Sort by date (most recent first)
master_df = master_df.sort_values('date', ascending=False)

# Rename date column to 'Date'
master_df = master_df.rename(columns={'date': 'Date'})

# Reset index
master_df = master_df.reset_index(drop=True)

## Data Filtering

In [ ]:
print(f"\nBefore filtering:")
print(f"Total rows: {len(master_df)}")

# Identify important columns (all except Date and US_CPI)
important_cols = [col for col in master_df.columns if col not in ['Date', 'US_CPI']]

# Apply improved filtering logic
print(f"\nApplying improved filtering logic...")

# Create a mask for rows worth keeping
keep_mask = master_df.apply(lambda row: is_row_worth_keeping(row, important_cols, min_important_values=8), axis=1)

# Filter the dataframe
master_df_filtered = master_df[keep_mask].copy()

print(f"\nAfter improved filtering (minimum 8 important non-null values, excluding CPI-only rows):")
print(f"Total rows: {len(master_df_filtered)}")
print(f"Rows removed: {len(master_df) - len(master_df_filtered)}")

# Show some statistics about the filtering
print(f"\nFiltering statistics:")
print(f"- Rows with only CPI data: {len(master_df[(master_df[important_cols].notna().sum(axis=1) == 0) & (master_df['US_CPI'].notna())])}")
print(f"- Rows with 1-7 important values: {len(master_df[(master_df[important_cols].notna().sum(axis=1) >= 1) & (master_df[important_cols].notna().sum(axis=1) < 8)])}")
print(f"- Rows with 8+ important values: {len(master_df[master_df[important_cols].notna().sum(axis=1) >= 8])}")

## Data Quality Check

In [ ]:
print("\nData Quality Check - Detailed analysis of filtered data:")
check_cols = [col for col in master_df_filtered.columns if col != 'Date']
important_cols_check = [col for col in check_cols if col != 'US_CPI']

print(f"\nNon-null value distribution in filtered data:")
non_null_counts = master_df_filtered[check_cols].notna().sum(axis=1)
important_non_null_counts = master_df_filtered[important_cols_check].notna().sum(axis=1)

print(f"- Total non-null values per row: min={non_null_counts.min()}, max={non_null_counts.max()}, mean={non_null_counts.mean():.1f}")
print(f"- Important non-null values per row: min={important_non_null_counts.min()}, max={important_non_null_counts.max()}, mean={important_non_null_counts.mean():.1f}")

# Check for any remaining sparse rows
sparse_rows = []
for idx, row in master_df_filtered.iterrows():
    non_null = row[check_cols].notna().sum()
    important_non_null = row[important_cols_check].notna().sum()
    if important_non_null < 8:
        sparse_rows.append((row['Date'], important_non_null, non_null))

if sparse_rows:
    print(f"\n⚠️  Found {len(sparse_rows)} rows with fewer than 8 important non-null values:")
    for date, important_count, total_count in sparse_rows:
        print(f"  {date}: {important_count} important, {total_count} total non-null values")
else:
    print(f"\n✓ All rows have at least 8 important non-null values!")

# Show population growth data alignment check
print(f"\nPopulation Growth Data Check:")
pop_growth_data = master_df_filtered[['Date', 'US_Population_Growth_Rate_FRED']].dropna()
print(f"- Population growth data points: {len(pop_growth_data)}")
if len(pop_growth_data) > 0:
    print(f"- Date range: {pop_growth_data['Date'].min()} to {pop_growth_data['Date'].max()}")
    print(f"- Sample values: {pop_growth_data.head(3)['US_Population_Growth_Rate_FRED'].tolist()}")

## Summary

In [ ]:
print(f"\n{'='*60}")
print(f"Master DataFrame Summary:")
print(f"{'='*60}")
print(f"Original total rows: {len(master_df)}")
print(f"Filtered total rows: {len(master_df_filtered)}")
print(f"Total columns: {len(master_df_filtered.columns)} (Date + {len(master_df_filtered.columns)-1} variables)")
print(f"Date range: {master_df_filtered['Date'].min()} to {master_df_filtered['Date'].max()}")
print(f"\nColumns: {', '.join(master_df_filtered.columns.tolist())}")

In [ ]:
master_df.head()

In [ ]:
master_df.info()

In [ ]:
master_df.describe()

### Save Output

In [ ]:
output_dir = 'data/processed'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Save to CSV
output_file = os.path.join(output_dir, 'bloomberg_master_dataframe.csv')
master_df_filtered.to_csv(output_file, index=False)
print(f"Master dataframe saved to: {output_file}")

In [ ]:
# Save to Excel
output_excel = os.path.join(output_dir, 'bloomberg_master_dataframe.xlsx')
master_df_filtered.to_excel(output_excel, index=False)
print(f"Master dataframe saved to: {output_excel}")